In [20]:
import configparser
from pathlib import Path
import collections
import pandas as pd
import pandas.io.sql as pandasql
from psycopg2.extras import execute_values
from psycopg2 import connect
import numpy
import matplotlib as mpl
from IPython.display import HTML, display
import rick
import numpy as np
import matplotlib.pyplot as plt
from setuptools import setup, find_packages
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [41]:
merge_set = pd.read_sql('''select * from congestion.mergeshort_result3_ordered_v2
                          ''',con)

In [22]:
def check_usage(link_set, link_used):
    result = []
    if link_used == []:
        return False
    for i in link_used:
        if collections.Counter(link_set) == collections.Counter(i):
            result.append(True)
        else:
            result.append(False)
    if True in result:
        return True
    else:
        return False

In [23]:
def make_selection(segment_set, min_angle, angle_diff, length_tolerance):
    ## find out number of ranks
    if pd.isna(segment_set['ang_diff'].iloc[0]):
        selected_rank = 0
    else:
        if segment_set['ang_diff'].iloc[0] < min_angle:
            if segment_set.shape[0] == 2:
                rank1=segment_set[(segment_set['rank']==1)]
                rank2=segment_set[(segment_set['rank']==2)]

                if rank2['ang_diff'].iloc[0] - rank1['ang_diff'].iloc[0] < angle_diff and rank2['pot_length'].iloc[0] - rank1['pot_length'].iloc[0] < length_tolerance:
                    selected_rank = 2
                else:
                    selected_rank = 1
            elif segment_set.shape[0] == 1:
                    selected_rank = 1
        else:
            selected_rank = 0
       
    return selected_rank

In [24]:
def return_chosen_segments(segment_set, rank, link_used):
    if rank != 0:
        whole_set = segment_set[(segment_set['rank']==rank)].iloc[0]
        original_set = whole_set[['start_vid', 'end_vid', 'segment_id', 'link_set', 'length_set', 'length']]
        additional_set = whole_set[['pot_start_vid', 'pot_end_vid', 'pot_seg', 'pot_link_set', 'pot_length_set','pot_length']]
        additional_links = additional_set['pot_link_set']
        if check_usage(additional_links, link_used) == False:
            # find which node to use for new start and end_vid
            if original_set['start_vid'] == additional_set['pot_end_vid']:
                new_start_vid = additional_set['pot_start_vid'].astype(float)
                new_end_vid = original_set['end_vid'].astype(float)
                new_link_set = additional_set['pot_link_set'] + original_set['link_set']
                new_length_set = additional_set['pot_length_set'] + original_set['length_set']
            else:
                new_start_vid = original_set['start_vid'].astype(float)
                new_end_vid = additional_set['pot_end_vid'] .astype(float)
                new_link_set = original_set['link_set'] + additional_set['pot_link_set']
                new_length_set = original_set['length_set'] + additional_set['pot_length_set']
            new_length = additional_set['pot_length'].astype(float) + original_set['length'].astype(float)
            original_link_set = original_set['link_set']
            additional_links = additional_set['pot_link_set']
        else:
            whole_set = segment_set.iloc[0]
            new_start_vid = original_set['start_vid'].astype(float)
            new_end_vid = original_set['end_vid'].astype(float)
            new_length = original_set['length'].astype(float)
            new_length_set = original_set['length_set']
            new_link_set = original_set['link_set']
            additional_links = []
            original_link_set = original_set['link_set']
            
    else:
        whole_set = segment_set.iloc[0]
        original_set = whole_set[['start_vid', 'end_vid', 'segment_id', 'link_set', 'length_set','length']]
        new_start_vid = original_set['start_vid'].astype(float)
        new_end_vid = original_set['end_vid'].astype(float)
        new_length = original_set['length'].astype(float)
        new_length_set = original_set['length_set']
        new_link_set = original_set['link_set']
        additional_links = []
        original_link_set = original_set['link_set']
    return new_start_vid, new_end_vid, new_length, new_link_set, new_length_set, additional_links, original_link_set

In [42]:
link_used = []
new_sets = []
rows = []
unique_set = merge_set[['segment_id']].drop_duplicates()

for index, key in unique_set.reset_index(drop=True).iterrows(): 
    segment_id = unique_set['segment_id'].iloc[index]
    prepare_set = merge_set[(merge_set['segment_id'] == segment_id)]
    link_set = prepare_set['link_set'].iloc[0]
    # See if the link_set was used before
    if check_usage(link_set, link_used) == False:
        # See if the link_set needs merging     
        if prepare_set['length'].iloc[0] < 200:
            rank = make_selection(prepare_set, 25, 5, 50)
            new_start_vid, new_end_vid, new_length, new_link_set, new_length_set, additional_links, original_link_set = return_chosen_segments(prepare_set, rank, link_used)
            if len(additional_links) != 0:  
                link_used.append(original_link_set)
                link_used.append(additional_links)
            else: 
                link_used.append(original_link_set)
            row = (new_start_vid, new_end_vid, new_link_set, new_length_set, new_length)
            rows.append(row)
        # Else append                                                             
        elif prepare_set['length'].iloc[0] > 200:
            new_start_vid, new_end_vid, new_length, new_link_set, new_length_set, additional_links, original_link_set = return_chosen_segments(prepare_set, 0, link_used)
            link_used.append(original_link_set)
            row = (new_start_vid, new_end_vid, new_link_set, new_length_set, new_length)
            rows.append(row)
    else:
        continue

In [44]:
sql = '''insert into congestion.mergeshort_result4_v2(start_vid, end_vid, link_set, length_set, length) VALUES %s'''    
with con:
    with con.cursor() as cur:
        execute_values(cur, sql, rows)    

In [31]:
def create_insert_table (table_version):
    with con:
        with con.cursor() as cur:
            cur.execute("create table congestion.mergeshort_result{}_v2 (start_vid bigint, end_vid bigint, link_set bigint[], length_set float[], length integer)".format(table_version))

In [43]:
create_insert_table(4)